Analyzing my personal strava data to estimate my future race performace

Questions I want to answer:
- Have I gotten faster
- Does the time I start running affect performace
- Do hills / elevation truly affect my speed
- Is strava miles to calorie calculation correct / consistent
- Can I build a accurate model to predict my runs?


In [4]:
import pandas as pd
import numpy as np

Data was downloaded from https://www.strava.com/athlete/download_request, unziped, and light data exploartion showed that out of the 38 csv files activities.csv held the raw data I needed. 

In [33]:

# load activities CSV (adjust filename if needed)
df = pd.read_csv('activities.csv')
dfM = df.copy()

In [34]:
df

,Activity ID,Activity Date,Activity Name,Activity Type,Activity Description,Elapsed Time,Distance,Max Heart Rate,Relative Effort,Commute,...,Intensity,Average Grade Adjusted Pace,Timer Time,Total Cycles,Recovery,With Pet,Competition,Long Run,For a Cause,Media
0,13124889853,"Dec 15, 2024, 11:29:06 PM",Prelude,Run,"Ate 5 cookies and felt guilty, but I AM BACK",6329,16.36,NaN,NaN,False,...,NaN,3.159,NaN,NaN,NaN,NaN,NaN,NaN,NaN,media/56A56CAA-7537-47AB-A764-31B45AA6B141.jpg...
1,13178383456,"Dec 24, 2024, 1:57:27 AM",Fat man run,Run,NaN,3638,4.82,NaN,NaN,False,...,NaN,1.528,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,13196474382,"Dec 26, 2024, 1:50:22 PM",Mountain hike,Run,NaN,1099,1.02,NaN,NaN,False,...,NaN,2.627,NaN,NaN,NaN,NaN,NaN,NaN,NaN,media/C760A05A-1389-4EB6-96FC-97A38DC2DA67.jpg...
3,13257314504,"Jan 3, 2025, 5:55:53 PM",Light run with tyan,Run,road to 500mi !!!,2452,6.51,NaN,NaN,False,...,NaN,2.850,NaN,NaN,NaN,NaN,NaN,NaN,NaN,media/6CD016E7-1097-495E-906E-897C0E209567.jpg...
4,13600552223,"Feb 11, 2025, 3:43:33 PM",Lite ting,Run,"First run in new shoes, took a while tp get us...",3102,6.97,NaN,NaN,False,...,NaN,2.652,NaN,NaN,NaN,NaN,NaN,NaN,NaN,media/7E33D434-EC5A-4397-A206-D17D8BB5E946.jpg...
5,13634498697,"Feb 15, 2025, 2:05:50 PM",Mr. Moiz Marathon,Run,very light campus run cuz greenway was FLOODED,3085,7.93,NaN,NaN,False,...,NaN,2.685,NaN,NaN,NaN,NaN,NaN,NaN,NaN,media/374B56FE-48C7-4437-A62C-CB700012C939.jpg...
6,13879572401,"Mar 14, 2025, 11:11:38 AM",Denis and Friend Jog,Run,"had to stop a ton cuz of anton, but still had ...",3450,9.14,NaN,NaN,False,...,NaN,2.759,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,14233461921,"Apr 20, 2025, 1:52:14 PM",Ruin Run,Run,Lot of fun!,1461,4.96,NaN,NaN,False,...,NaN,3.892,NaN,NaN,NaN,NaN,NaN,NaN,NaN,media/07F23CB2-8F38-4C4D-86BF-9B2D0ABB3EF4.jpg...
8,14322705991,"Apr 29, 2025, 2:34:44 PM",warmup to next post,Run,NaN,497,1.78,NaN,NaN,False,...,NaN,3.708,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,14322809190,"Apr 29, 2025, 2:47:24 PM",85% effort mile,Run,"honestly, I’m pretty surprised at how fast I a...",395,1.83,NaN,NaN,False,...,NaN,4.579,NaN,NaN,NaN,NaN,NaN,NaN,NaN,media/2D735961-B2A4-4F5E-9410-5C48F8515960.jpg...


df

In [37]:
# data cleaning
# Cols: 	Activity ID	Activity Date	Activity Name	Activity Type	Activity Description	Elapsed Time	Distance	Max Heart Rate	Relative Effort	Commute	...	Intensity	Average Grade Adjusted Pace	Timer Time	Total Cycles	Recovery	With Pet	Competition	Long Run	For a Cause	Media

# drop cols that are not needed by selecting relevant ones

df = dfM.copy()

relevant_cols = [
    'Activity Date', 'Activity Name', 'Activity Type',
    'Elapsed Time', 'Distance', 'Moving Time', 
    'Elevation Gain', 'Elevation Loss', 'Calories', 
    'Grade Adjusted Distance', 'Total Steps', 'Average Grade Adjusted Pace'
]
df = df[relevant_cols]

# convert 'Activity Date' to datetime
df['Activity Date'] = pd.to_datetime(df['Activity Date'])

# Filter for only runs
df = df[df['Activity Type'] == 'Run']
df.head()

# Remove the copy of colmuns for the extra  (Elapsed Time	Elapsed Time	Elapsed Time	Elapsed Time	Elapsed Time	Elapsed Time	Elapsed Time)

df = df.loc[:,~df.columns.duplicated()]

df.head()


# convert distance from km to miles

df['Distance'] = df['Distance'] * 0.621371

df.head()

C:\Users\shaan\AppData\Local\Temp\ipykernel_32596\2329705816.py:17: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['Activity Date'] = pd.to_datetime(df['Activity Date'])


,Activity Date,Activity Name,Activity Type,Elapsed Time,Distance,Moving Time,Elevation Gain,Elevation Loss,Calories,Grade Adjusted Distance,Total Steps,Average Grade Adjusted Pace
0,2024-12-15 23:29:06,Prelude,Run,6329,10.165630,5227.0,127.5,127.5,1472.0,16509.7,13970.0,3.159
1,2024-12-24 01:57:27,Fat man run,Run,3638,2.995008,3201.0,40.5,40.5,436.0,4891.8,6832.0,1.528
2,2024-12-26 13:50:22,Mountain hike,Run,1099,0.633798,663.0,124.6,124.6,155.0,1741.6,1586.0,2.627
3,2025-01-03 17:55:53,Light run with tyan,Run,2452,4.045125,2308.0,52.9,52.9,587.0,6578.3,6234.0,2.850
4,2025-02-11 15:43:33,Lite ting,Run,3102,4.330956,2660.0,36.4,36.4,629.0,7053.3,4814.0,2.652


In [ ]:
# - Have I gotten faster




